In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score,classification_report

In [ ]:
df = pd.read_csv('/content/heart_disease_data.csv')
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [ ]:
df.dropna

<bound method DataFrame.dropna of      age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
0     63    1   3       145   233    1        0      150      0      2.3   
1     37    1   2       130   250    0        1      187      0      3.5   
2     41    0   1       130   204    0        0      172      0      1.4   
3     56    1   1       120   236    0        1      178      0      0.8   
4     57    0   0       120   354    0        1      163      1      0.6   
..   ...  ...  ..       ...   ...  ...      ...      ...    ...      ...   
298   57    0   0       140   241    0        1      123      1      0.2   
299   45    1   3       110   264    0        1      132      0      1.2   
300   68    1   0       144   193    1        1      141      0      3.4   
301   57    1   0       130   131    0        1      115      1      1.2   
302   57    0   1       130   236    0        0      174      0      0.0   

     slope  ca  thal  target  
0        0   0     1       1  
1        0   0     2       1  
2        2   0     2       1  
3        2   0     2       1  
4        2   0     2       1  
..     ...  ..   ...     ...  
298      1   0     3       0  
299      1   0     3       0  
300      1   2     3       0  
301      1   1     3       0  
302      1   1     2       0  

[303 rows x 14 columns]>

In [ ]:
df.shape

(303, 14)

In [ ]:
df.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In target values:

   1: Healthy Heart

   0: Defective Heart

In [ ]:
Y = df['target']

In [ ]:
Y

,target
0,1
1,1
2,1
3,1
4,1
...,...
298,0
299,0
300,0
301,0


In [ ]:
X = df.drop(columns = 'target',axis=1)

# SPLITTING DATA

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size =0.27, random_state = 23)

In [ ]:
print(X_test.shape)
print(X_train.shape)

(82, 13)
(221, 13)


# CHOOSING AND TRAINING AN ML MODEL

In [ ]:
models = {
    'LogisticRegression': (LogisticRegression(max_iter=1000), {
        'C': [0.001, 0.01, 0.1, 1, 10],
        'solver': ['liblinear', 'saga']
    }),
    'RandomForest': (RandomForestClassifier(), {
        'n_estimators': [10, 50, 100],
        'max_depth': [None, 10, 20, 30]
    }),
    'DecisionTree': (DecisionTreeClassifier(), {
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10]
    }),
    'XGBoost': (xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss'), {
        'n_estimators': [50, 100, 200],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1, 0.2]
    })
}

In [ ]:
best_models = {}
for model_name, (model, params) in models.items():
    print(f"Searching best parameters for {model_name}...")
    grid_search = GridSearchCV(model, params, cv=5, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_train, Y_train)

    # Store the best model
    best_models[model_name] = grid_search.best_estimator_
    print(f"Best parameters for {model_name}: {grid_search.best_params_}")

Searching best parameters for LogisticRegression...
Best parameters for LogisticRegression: {'C': 0.1, 'solver': 'liblinear'}
Searching best parameters for RandomForest...
Best parameters for RandomForest: {'max_depth': None, 'n_estimators': 100}
Searching best parameters for DecisionTree...
Best parameters for DecisionTree: {'max_depth': 20, 'min_samples_split': 10}
Searching best parameters for XGBoost...
Best parameters for XGBoost: {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 100}


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [10:07:54] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [ ]:
for model_name, best_model in best_models.items():
    Y_pred = best_model.predict(X_test)
    accuracy = accuracy_score(Y_test, Y_pred)
    print(f"{model_name} accuracy: {accuracy:.4f}")
    print(classification_report(Y_test, Y_pred))

LogisticRegression accuracy: 0.8659
              precision    recall  f1-score   support

           0       0.94      0.78      0.85        40
           1       0.82      0.95      0.88        42

    accuracy                           0.87        82
   macro avg       0.88      0.86      0.86        82
weighted avg       0.88      0.87      0.86        82

RandomForest accuracy: 0.8659
              precision    recall  f1-score   support

           0       0.89      0.82      0.86        40
           1       0.84      0.90      0.87        42

    accuracy                           0.87        82
   macro avg       0.87      0.86      0.87        82
weighted avg       0.87      0.87      0.87        82

DecisionTree accuracy: 0.7317
              precision    recall  f1-score   support

           0       0.70      0.80      0.74        40
           1       0.78      0.67      0.72        42

    accuracy                           0.73        82
   macro avg       0.74      0.7

So we'll be training a Random Forest Classifier as it has 86.6% accuracy, the highest available.

In [ ]:
model = RandomForestClassifier(max_depth = 30, n_estimators = 100)
model.fit(X_train,Y_train)

RandomForestClassifier(max_depth=30)

In [ ]:
Y_pred = model.predict(X_test)

In [ ]:
score = accuracy_score(Y_test,Y_pred)

In [ ]:
print("Accuracy score of Random Forest Classifier is",score)

Accuracy score of Random Forest Classifier is 0.8292682926829268


So the optimal parameter for Random Forest max_depth = None wasn't implementable, hence the accuracy dropped since we were using another value. Let'strain a Logistic Regression Model since it also has 86.6% accuracy.

In [ ]:
model = LogisticRegression(C=0.1, solver='liblinear')
model.fit(X_train,Y_train)

LogisticRegression(C=0.1, solver='liblinear')

In [ ]:
Y_pred = model.predict(X_test)

In [ ]:
score = accuracy_score(Y_pred,Y_test)
print("Accuracy score of the Logistic Regression Model is", score)

Accuracy score of the Logistic Regression Model is 0.8658536585365854


Lessgoooo....86.59% accuracy. Great job.

# BUILDING A PREDICTIVE SYSTEM

In [ ]:
input = (62,0,0,40,268,0,0,160,0,3.6,0,2,2)
arr = np.array(input)
arr

array([ 62. ,   0. ,   0. ,  40. , 268. ,   0. ,   0. , 160. ,   0. ,
         3.6,   0. ,   2. ,   2. ])

In [ ]:
prediction = model.predict(arr.reshape(1,-1))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [ ]:
if prediction == 1:
  print("You are healthy :)")
else:
  print("Please consult a doctor")

You are healthy :)
